# World War 2 Bombings
This tutorial provides two methods of creating the visualization for example two, both of which pertain to usage within Python code.

Note that this needs to be run three different times for each .csv file (for each year).

### Method 2a: Using GeoHexSimple's functions to run a properties file

In [ ]:
from geohexviz.utils.file import run_file

# For each run, in the json_structure.json file, change the hexbin layer data path
# start with it set to data-1943.csv, then data-1944.csv, and finally data-1945.csv.
run_file("json_structure.json") # JSON file works

Simple enough, right?

### Method 2b: Using GeoHexViz's functions to make a plot from scratch

In [ ]:
from geohexviz.builder import PlotBuilder

After our import, we need to instantiate a PlotBuilder object.
This object contains the properties, and layers of the plot.

In [ ]:
myBuilder = PlotBuilder()

Now we must set the properties of the required layer (referred to as the hexbin layer).
This layer contains the hexagonally binned data.
Note how the ```marker.line.width``` property for this layer is being changed.
This changed the thickness of the lines defining each hexagon on the plot.
This will be important later.

In [ ]:
myBuilder.set_hexbin(
    data="data-1943.csv",           # remember to change this every run (data-1943.csv, data-1944.csv, data-1945.csv)
    hexbin_info=dict(
        binning_field="high_explosives_weight_tons",
        binning_fn="sum"
    ),
    hex_resolution=4,
    manager=dict(
        marker=dict(
            line=dict(width=0.45)
        ),
        colorscale="Viridis",
        colorbar=dict(
            x=0.82
        )
    )
)            

Now we add a region-type layer containing the Europe.

Note: GeoHexViz and GeoHexSimple alike recognize the names of countries and continents, so no file is needed.

In [ ]:
myBuilder.add_region(
    name="sample_Region_EUROPE",
    data="EUROPE"
)

For this example, we want a continuous grid so we add a grid-type layer for Europe.

In [ ]:
myBuilder.add_grid(
    name="sample_Grid_EUROPE",
    data="EUROPE",
    convex_simplify=True
)

Since the region of Europe is so large and it is semi-close to the North Pole, GeoHexViz has issues generating the grid over it. 
When visualized, it seems that there are limited hexagons over Russia and some hexagons are in odd locations.
To combat this, we add a grid layer over Russia, and later we clip the grid layers to the region layers.

In [ ]:
myBuilder.add_grid(
    name="sample_Grid_RUSSIA",
    data="RUSSIA",
    convex_simplify=True
)

Next, since the ```marker.line.width``` property was changed for the hexbin layer earlier, we must change this property for both grid layers.
To do this, we have to access the manager for grid layers.

In [ ]:
myBuilder.update_grid_manager(
    marker=dict(
        line=dict(width=0.45)
    )
)

For this example in particular, it becomes apparent that when visualizing, the ```adjust_focus``` function does not provide an adequate focus for the plot (not easily at least). 
To combat this, we need to access the properties of the plot itself.
We set the figure's ```lataxis.range```, ```lonaxis.range```, and ```projection.rotation``` properties.

In [ ]:
myBuilder.update_figure(
    geos=dict(
        lataxis=dict(
            range=[35, 58]
        ),
        lonaxis=dict(
            range=[0, 43]
        ),
        projection=dict(
            rotation=dict(
                lat=46.63321662159487,
                lon=11.21560455920799
            )
        )
    )
)

Now, we invoke functions on the stored layers. For this example, since the focus is on Europe, we want to clip the hexbin layer, and grid layers to Europe. We also want the plot to use a logarithmic scale.

In [ ]:
myBuilder.clip_layers(
    clip="hexbin+grids",
    to="regions"
)
myBuilder.logify_scale(
    exp_type="r"
)
# * Unlike JSON input mechanism, in a module adjust\_focus is not
# * invoked by default, the user has to invoke it

Next we finalize/output the visualization.

In [ ]:
myBuilder.finalize()

 # remember to change this every run:
 # output_visualization-1943.pdf, output_visualization-1944.pdf, output_visualization-1945.pdf

myBuilder.output(
    filepath="output_visualization-1943.pdf",
    crop_output=True
)